## Setting up the environment

In [1]:
# move to the project directory
import os
os.chdir("..")

from pathlib import Path
MOLECULES_FILE = 'molecules.json'
MOLECULES_DIR = Path('.', 'data', MOLECULES_FILE)

import pandas as pd
from pprint import pprint

# import the project
from quantum_simulation.drivers import molecule_loader

In [2]:
molecules = molecule_loader.load_molecule(MOLECULES_DIR)
pprint(molecules)

[MoleculeInfo(symbols=['H', 'H'],
              coords=[[0.0, 0.0, 0.0], [0.0, 0.0, 0.74]],
              multiplicity=1,
              charge=0,
              units=<DistanceUnit.ANGSTROM: 'Angstrom'>,
              masses=[1.008, 1.008])]


### See results from the IBM Quantum

In [9]:
from qiskit_ibm_runtime import QiskitRuntimeService
import os

service = QiskitRuntimeService(
    channel=os.getenv('IBM_RUNTIME_CHANNEL'),
    instance=os.getenv('IBM_RUNTIME_INSTANCE'),
    token=os.getenv('IBM_RUNTIME_TOKEN')
)
job = service.job('cy5seka01rbg008j62mg')
job_result = job.result()

for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")

Expectation values for pub 0: [-1.06959996]
